In [1]:
#imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import sys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
#Driver for firefox and loading the Eweb url
driver = webdriver.Firefox()
driver.implicitly_wait(10)

driver.get("https://www.easternct.edu/faculty-directory/index.html")

In [3]:
#selecting SearchButton
Search = driver.find_element(by = 'id', value = 'directorySearchButton')
Search.click()

In [4]:
#Locating the HTML where the info is stored
table = driver.find_elements(by = 'class name', value = "mix")

In [5]:
print(len(table))

876


In [6]:
for t in table[1:]:
    print(t.text)
    print()

In [ ]:
#Close the webpage
driver.quit()

In [8]:
def parse_professor_info(text):
    """
    Parse professor info from a multi-line string into a dictionary.
    Expected format:
      Line 1: Name
      Line 2: Position
      Line 3: Department
      Subsequent lines: 'Phone:', 'Email:', 'Office:' and optionally 'Hours:'
    """
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    professor = {
        'name': None,
        'position': None,
        'department': None,
        'phone': None,
        'email': None,
        'office': None,
        'Hours': None
    }
    
    if len(lines) >= 1:
        professor['name'] = lines[0]
    if len(lines) >= 2:
        professor['position'] = lines[1]
    if len(lines) >= 3:
        professor['department'] = lines[2]
    
    for line in lines:
        if line.startswith('Phone:'):
            professor['phone'] = line.replace('Phone:', '').strip()
        elif line.startswith('Email:'):
            professor['email'] = line.replace('Email:', '').strip()
        elif line.startswith('Office:'):
            professor['office'] = line.replace('Office:', '').strip()
        elif line.startswith('Hours:'):
            professor['Hours'] = line.replace('Hours:', '').strip()
    
    return professor

def main():
    # Load the CSV file with extracted text
    df = pd.read_csv("output.csv")
    
    # Parse each row and create a new DataFrame with structured data
    parsed_data = df["Extracted Text"].apply(parse_professor_info).tolist()
    parsed_df = pd.DataFrame(parsed_data)
    
    # Save the structured DataFrame to a new CSV file
    parsed_df.to_csv("AllFacultyInfo.csv", index=False)
    print("Parsed CSV saved as 'AllFacultyInfo.csv'")

if __name__ == '__main__':
    main()

Parsed CSV saved as 'AllFacultyInfo.csv'


In [11]:
#New
import csv

# Open the input CSV file
with open('AllFacultyInfo.csv', mode='r', newline='', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    
    # Filter the rows where 'Hours' field is not empty
    filtered_rows = [row for row in reader if row['Hours'].strip() != '']
    
    # Specify the output CSV file
    with open('Exclusive_Hours.csv', mode='w', newline='', encoding='utf-8') as outfile:
        fieldnames = reader.fieldnames
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
        
        # Write the filtered rows
        writer.writerows(filtered_rows)

print("Filtered data has been saved to 'Exclusive_Hours.csv'")

Filtered data has been saved to 'Exclusive_Hours.csv'


In [15]:
# Open the Exclusive_Hours.csv file
with open('Exclusive_Hours.csv', mode='r', newline='', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    
    # Specify the output CSV file for the new format
    with open('Formatted_Exclusive_Hours.csv', mode='w', newline='', encoding='utf-8') as outfile:
        # Define the new fieldnames (columns)
        fieldnames = ['name', 'office', 'Hours']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
        
        # Process each row and format the name as "last name, first initial"
        for row in reader:
            name_parts = row['name'].split()  # Split the name by spaces
            if len(name_parts) >= 2:
                last_name = name_parts[-1]
                first_initial = name_parts[0][0]  # Get the first initial of the first name
                formatted_name = f"{last_name}, {first_initial}"
            else:
                formatted_name = row['name']  # If the name doesn't split properly, just keep it as is
                
            # Write the formatted row
            writer.writerow({
                'name': formatted_name,
                'office': row['office'],
                'Hours': row['Hours']
            })

print("Formatted data has been saved to 'Formatted_Exclusive_Hours.csv'")

Formatted data has been saved to 'Formatted_Exclusive_Hours.csv'
